import

In [1]:
%load_ext autoreload
%autoreload 2
import json
import dgl
from sklearn.metrics import f1_score
import torch as th
import dgl.nn.pytorch as dglnn
import torch.nn as nn
from dgl.dataloading import GraphDataLoader
import torch.nn.functional as F
import networkx as nx
import warnings
from feature_extraction.base_featurizer import BaseFeaturizer
import torch
from graph_construction.tps_graph import create_dummy_dgl_graph, tps_graph
import numpy as np

from dgl_classifier.trainer import GraphDataset,snap_pred, tps_to_dgl, get_clasification_vec,\
tps_graph_const, extract_data, runner


import warnings

from preprocessing.utils import load_BGPS_from_json
warnings.filterwarnings('ignore', category=UserWarning, message='TypedStorage is deprecated')

In [2]:
#seed
dgl.seed(1223)

Model

In [3]:
class Classifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes):
        super(Classifier, self).__init__()
        self.conv1 = dglnn.RelGraphConv(in_dim, hidden_dim, 8)
        #self.conv1 = dglnn.GraphConv(in_dim, hidden_ˇdim)
        self.conv2 = dglnn.RelGraphConv(hidden_dim, hidden_dim, 8)
        #self.conv2 = dglnn.GraphConv(hidden_dim, hidden_dim)
        self.classify = nn.Linear(hidden_dim, n_classes)

    def forward(self, g, h, rel_types):
        # Apply graph convolution and activation.
        h = F.relu(self.conv1(g, h, rel_types))
        h = F.relu(self.conv2(g, h, rel_types))
        with g.local_scope():
            g.ndata['node_features'] = h
            # Calculate graph representation by average readout.
            hg = dgl.mean_nodes(g, 'node_features')
            return F.softmax( self.classify(hg), dim=1)
model = Classifier(113, 20, 2)

hyperparameters

In [4]:
early_stop =10
model = Classifier(113, 20, 2)
lr = 0.01
wd = 5e-4
epochs = 100

file paths

In [6]:
train_file = '/work/data/splits/train.json'
val_file = '/work/data/splits/val.json'
test_file = '/work/data/splits/test.json'
path_to_save = '/work/data/models'
path_to_res = '/work/data/results/25_5_2023_tp_graph_1re.txt'

In [7]:
train_dataloader, val_dataloader, test_dataloader = extract_data(train_file, val_file, test_file, community_no=30, batch_size = 50, verbose=False)

/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/tensor.py:52: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return th.as_tensor(data, dtype=dtype)


In [8]:
runner(train_dataloader,val_dataloader,test_dataloader,model, early_stop, lr, wd, epochs, path_to_res)

Epoch 1
--------------------------------------------------------------
Epoch:    1        1 Batch loss: 0.704280 Batch F1: 0.11764705882352941
Epoch:    1        2 Batch loss: 0.683773 Batch F1: 0.5306122448979592
Epoch:    1        3 Batch loss: 0.664188 Batch F1: 0.6349206349206348
Epoch:    1        4 Batch loss: 0.650744 Batch F1: 0.7666666666666667
Epoch:    1        5 Batch loss: 0.632558 Batch F1: 0.7
Epoch:    1        6 Batch loss: 0.556272 Batch F1: 0.8571428571428571
Train Avg Loss    1: 0.648636

Train Avg F1    1: 0.601164910408608

Val Avg Loss    1: 0.207119

Val Avg F1    1:  0.6329411764705883

Optimal Val loss (Epoch 1): 0.20711884647607803

Epoch 2
--------------------------------------------------------------
Epoch:    2        1 Batch loss: 0.637679 Batch F1: 0.5957446808510638
Epoch:    2        2 Batch loss: 0.628127 Batch F1: 0.5333333333333333
Epoch:    2        3 Batch loss: 0.536864 Batch F1: 0.7200000000000001
Epoch:    2        4 Batch loss: 0.592621 Batch 